In [1]:
# mount google drive to colab for dataset
from google.colab import drive
drive.mount('/content/drive')

import os
os.chdir('/content/drive/MyDrive/DL_PROJECT')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.activity.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fexperimentsandconfigs%20https%3a%2f%2fwww.googleapis.com%2fauth%2fphotos.native&response_type=code

Enter your authorization code:
4/1AX4XfWgwHiO1PeNAwvjMzqYTOsRH9aoXWxHLeVtVaDGRhcdqesPWFadE8_E
Mounted at /content/drive


In [2]:
!pip install transformers

     |████████████████████████████████| 2.6 MB 7.6 MB/s 
     |████████████████████████████████| 636 kB 50.3 MB/s 
     |████████████████████████████████| 3.3 MB 38.4 MB/s 
     |████████████████████████████████| 895 kB 26.4 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [3]:
!pip install jsonlines

In [4]:
'''
tokenize all of the sentences and map the tokens to their word IDs.
'''
def tokenize(sequences):
    
    input_ids = []
    attention_masks = []

    # For every caption...
    for seq in sequences:
        '''
        `encode_plus` will:
          (1) Tokenize the caption.
          (2) Prepend the `[CLS]` token to the start.
          (3) Append the `[SEP]` token to the end.
          (4) Map tokens to their IDs.
          (5) Pad or truncate the sentence to `max_length`
          (6) Create attention masks for [PAD] tokens.
        '''
        encoded_dict = tokenizer.encode_plus(
                            seq,                       # Sentence to encode.
                            add_special_tokens = True, # Add '[CLS]' and '[SEP]'
                            max_length = 48,           # Pad & truncate all sentences.
                            truncation=True,
                            pad_to_max_length = True,
                            return_attention_mask = True,   # Construct attn. masks.
                            return_tensors = 'pt',      # Return pytorch tensors.
                       )

        # Add the encoded sentence to the list.    
        input_ids.append(encoded_dict['input_ids'])

        # And its attention mask (simply differentiates padding from non-padding).
        attention_masks.append(encoded_dict['attention_mask'])

    # Convert the lists into tensors.
    input_ids = torch.cat(input_ids, dim=0)
    attention_masks = torch.cat(attention_masks, dim=0)
    
    
    return input_ids, attention_masks

In [6]:
from PIL import Image
import json
import jsonlines
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader, Dataset, TensorDataset
from torch.utils import data
from torchvision import transforms, datasets, models
import matplotlib.pyplot as plt
import torch.optim as optim
from collections import defaultdict

#Load the BERT tokenizer.
import transformers
from transformers import BertTokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased', do_lower_case=True)

'''
Dataloader for creating predictions.csv
Returns (Image, Captions, Input_id, Attention_mask and ImageName)
'''
class mytestdataset():    

    def __init__(self, classification_list, name):

        super(mytestdataset).__init__()
        
        self.X = []
        self.Cap = []
        self.Imagename = []

        with jsonlines.open(classification_list) as f:
          f1 = []
          for line in f:
            f1.append(line)
          for line1 in f1:
            path =  line1['img']
            self.X.append('/content/drive/My Drive/DL_PROJECT/hateful_memes/hateful_memes/'+line1['img'])
            self.Cap.append(line1['text'])
            self.Imagename.append(line1['id'])
            #self.Imagename.append(path.split('/')[1][:-4]) 
        '''
        with open(classification_list, mode = 'r') as f:
            f1 = json.dumps(str(f))
            f2 = json.loads(f1)
            for line in f2.split('\t'):
            #for line in f.split('\t'):
                print(line)
                id, path, caption = json.loads(line)['id'],json.loads(line)['img'],json.loads(line)['text']
                #line = dict(list)
                #path = line["img"] #json.loads(line)["img"]
                self.X.append('/content/drive/My Drive/DL_PROJECT/hateful_memes/hateful_memes/'+path)
                self.Cap.append(caption)
                self.Imagename.append(id)
                #self.Imagename.append(path.split('/')[1][:-4])  
        '''  
        '''
        Tokenize all of the captions and map the tokens to their word IDs, and get respective attention masks.
        '''
        self.input_ids, self.attention_masks = tokenize(self.Cap)
        
        
        
        '''
        Image Transforms
        '''
        self.transform = transforms.Compose([   transforms.Resize(256),
                                                transforms.CenterCrop(224),
                                                transforms.ToTensor(),
                                                transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                                                     std=[0.229, 0.224, 0.225])
                                            ])
         
    
    def __getitem__(self,index):
        
        
        '''
        Image
        '''
        image = self.X[index]
                
        image = Image.open(image).convert('RGB') #.replace('img/', '')))
               
        image = self.transform(image)
        
       
        '''
        For Captions, Input ids, Attention mask and Imagename
        '''
        caption = self.Cap[index]
        input_id = self.input_ids[index]
        attention_masks = self.attention_masks[index]
        Imagename = self.Imagename[index]
        
        return image, caption, input_id, attention_masks, Imagename
        
  
    def __len__(self):
        return len(self.X)

'''
Dataloader for Training/Validation with support for Image Captioning model
Returns (Image, Caption, Input_id, Attention_mask, Input_id_Captioning_model, Attention_mask_Captioning_model, label)
'''
'''
class mydataset_captioning():    

    def __init__(self, classification_list, name):

        super(mydataset_captioning).__init__()
        
        self.X = []
        self.true_Cap = []
        self.generated_Cap = []
        self.Y = []
        
        with open(classification_list, mode = 'r') as f:

            for line in f:
                print(line[:-1].split('\n'))
                path, caption, generated_caption, label = line[:-1].split('\n')

                self.X.append('/content/drive/My Drive/DL_PROJECT/hateful_memes/hateful_memes/'+path)
                self.true_Cap.append(caption)
                self.generated_Cap.append(generated_caption)
                self.Y.append(label)
      
        #Tokenize all of the captions and map the tokens to thier word IDs, and get respective attention masks.

        self.input_ids, self.attention_masks = tokenize(self.true_Cap)
        
        self.input_ids_cap, self.attention_masks_cap = tokenize(self.generated_Cap)


        #Image Transforms

        
        if name in ['valid','test']:
            self.transform = transforms.Compose([   transforms.Resize(384),
                                                 transforms.CenterCrop(256),
                                                transforms.ToTensor(),
                                                transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                                                    std=[0.229, 0.224, 0.225])
                                                ])
        else:
            self.transform = transforms.Compose([ transforms.Resize(256),
                                                 transforms.RandomCrop(224),
                                                transforms.RandomHorizontalFlip(),
                                                transforms.ToTensor(),
                                                transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                                                    std=[0.229, 0.224, 0.225])
                                                                                            ])
    
    
    def __getitem__(self,index):

        #For Image and Label

        image = self.X[index]
                
        image = (Image.open(image))
               
        image = self.transform(image)
        
        label = float(self.Y[index])

        

        #For Captions, Input ids and Attention mask

        caption = self.true_Cap[index]
        input_id = self.input_ids[index]
        attention_masks = self.attention_masks[index]
            
        input_id_cap = self.input_ids_cap[index]
        attention_masks_cap = self.attention_masks_cap[index]
    
            
            
        return image, caption, input_id, attention_masks, input_id_cap, attention_masks_cap, torch.as_tensor(label).long()
        
  
    def __len__(self):
        return len(self.X)
'''
'''
Dataloader for Training/Validation
Returns (Image, Caption, Input_id, Attention_mask, label)
'''
class mydataset():    

    def __init__(self, classification_list, name):

        super(mydataset).__init__()
        
        self.X = []
        self.Cap = []
        self.Y = []

        with jsonlines.open(classification_list) as f:
          f1 = []
          for line in f:
            f1.append(line)
          for line1 in f1:
            path =  line1['img']
            self.X.append('/content/drive/My Drive/DL_PROJECT/hateful_memes/hateful_memes/'+line1['img'])
            self.Cap.append(line1['text'])
            self.Y.append(line1['label'])
            #self.Imagename.append(path.split('/')[1][:-4]) 
        '''
        with open(classification_list, mode = 'r') as f:
            
            for line in f:
                print(line)
                path, caption, label = line[1],line[3],line[2] #line[:-1].split('\t')

                self.X.append('/content/drive/My Drive/DL_PROJECT/hateful_memes/hateful_memes/'+path)
                self.Cap.append(caption)
                self.Y.append(label)
        '''
        '''
        Tokenize all of the captions and map the tokens to thier word IDs, and get respective attention masks.
        '''
        self.input_ids, self.attention_masks = tokenize(self.Cap)
        
        '''
        Image Transforms
        '''
        
        if name in ['valid','test']:
            self.transform = transforms.Compose([   transforms.Resize(384),
                                                 transforms.CenterCrop(256),
                                                transforms.ToTensor(),
                                                transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                                                    std=[0.229, 0.224, 0.225])
                                                ])
        else:
            self.transform = transforms.Compose([ transforms.Resize(256),
                                                 transforms.RandomCrop(224),
                                                transforms.RandomHorizontalFlip(),
                                                transforms.ToTensor(),
                                                transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                                                    std=[0.229, 0.224, 0.225])
                                                                                            ])
    
    
    def __getitem__(self,index):
        
        
        '''
        For Image and Label
        '''
        image = self.X[index]
                
        image = Image.open(image).convert('RGB') #.replace('img/', '')))
               
        image = self.transform(image)
        
        label = float(self.Y[index])

        
        '''
        For Captions, Input ids and Attention mask
        '''
        caption = self.Cap[index]
        input_id = self.input_ids[index]
        attention_masks = self.attention_masks[index]
        
        return image, caption, input_id, attention_masks, torch.as_tensor(label).long()
        
  
    def __len__(self):
        return len(self.X)

In [7]:
######### Load saved model from checkpoint  #########
def load(modelpath, model, optimizer, lr_scheduler):
    checkpoint = torch.load(modelpath)
    model.load_state_dict(checkpoint['model_state_dict'])
    optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
    train_loss = checkpoint['Training_Loss_List'] 
    v_loss = checkpoint['Validation_Loss_List']
    v_acc = checkpoint['Validation_Accuracy_List']

    epoch = checkpoint['Epoch']
    lr_scheduler.load_state_dict(checkpoint['lr_scheduler'])
    
    return model, optimizer, lr_scheduler, train_loss, v_loss, v_acc, epoch

In [8]:
def plot_loss(epochs, train_loss, v_loss, title):
    plt.figure(figsize=(8,8))
    x = np.arange(1,epochs+2)
    plt.plot(x, train_loss, label = 'Training Loss')
    plt.plot(x, v_loss, label = 'Validation Loss')
    plt.xlabel('Epochs', fontsize =16)
    plt.ylabel('Loss', fontsize =16)
    plt.title(title,fontsize =16)
    plt.legend(fontsize=16)
    
    
def plot_acc(epochs,v_acc):
    plt.figure(figsize=(8,8))
    x = np.arange(1,epochs+2)
    plt.plot(x, v_acc)
    plt.xlabel('Epochs', fontsize =16)
    plt.ylabel('Validation Accuracy', fontsize =16)
    plt.title('Validation Accuracy v/s Epochs',fontsize =16)

In [11]:
class BasicBlock(nn.Module):
    expansion = 1

    def __init__(self, inplanes, planes, stride=1, downsample=None, groups=1,
                 base_width=64, dilation=1, norm_layer=None):
        super(BasicBlock, self).__init__()
        if norm_layer is None:
            norm_layer = nn.BatchNorm2d
        if groups != 1 or base_width != 64:
            raise ValueError('BasicBlock only supports groups=1 and base_width=64')
        if dilation > 1:
            raise NotImplementedError("Dilation > 1 not supported in BasicBlock")
        # Both self.conv1 and self.downsample layers downsample the input when stride != 1
        self.conv1 = conv3x3(inplanes, planes, stride)
        self.bn1 = norm_layer(planes)
        self.relu = nn.ReLU(inplace=True)
        self.conv2 = conv3x3(planes, planes)
        self.bn2 = norm_layer(planes)
        self.downsample = downsample
        self.stride = stride

    def forward(self, x):
        identity = x

        out = self.conv1(x)
        out = self.bn1(out)
        out = self.relu(out)

        out = self.conv2(out)
        out = self.bn2(out)

        if self.downsample is not None:
            identity = self.downsample(x)

        out += identity
        out = self.relu(out)

        return out


class ResNet(nn.Module):

    def __init__(self, block, layers, num_classes=1000, zero_init_residual=False,
                 groups=1, width_per_group=64, replace_stride_with_dilation=None,
                 norm_layer=None):
        super(ResNet, self).__init__()
        if norm_layer is None:
            norm_layer = nn.BatchNorm2d
        self._norm_layer = norm_layer

        self.inplanes = 64
        self.dilation = 1
        if replace_stride_with_dilation is None:
            # each element in the tuple indicates if we should replace
            # the 2x2 stride with a dilated convolution instead
            replace_stride_with_dilation = [False, False, False]
        if len(replace_stride_with_dilation) != 3:
            raise ValueError("replace_stride_with_dilation should be None "
                             "or a 3-element tuple, got {}".format(replace_stride_with_dilation))
        self.groups = groups
        self.base_width = width_per_group
        self.conv1 = nn.Conv2d(3, self.inplanes, kernel_size=7, stride=2, padding=3,
                               bias=False)
        self.bn1 = norm_layer(self.inplanes)
        self.relu = nn.ReLU(inplace=True)
        self.maxpool = nn.MaxPool2d(kernel_size=3, stride=2, padding=1)
        self.layer1 = self._make_layer(block, 64, layers[0])
        self.layer2 = self._make_layer(block, 128, layers[1], stride=2,
                                       dilate=replace_stride_with_dilation[0])
        self.layer3 = self._make_layer(block, 256, layers[2], stride=2,
                                       dilate=replace_stride_with_dilation[1])
        self.layer4 = self._make_layer(block, 512, layers[3], stride=2,
                                       dilate=replace_stride_with_dilation[2])
        self.avgpool = nn.AdaptiveAvgPool2d((1, 1))
#         self.batchnorm = nn.BatchNorm1d(num_features=512 * block.expansion)
        self.fc = nn.Linear(512 * block.expansion, num_classes)

        for m in self.modules():
            if isinstance(m, nn.Conv2d):
                nn.init.kaiming_normal_(m.weight, mode='fan_out', nonlinearity='relu')
            elif isinstance(m, (nn.BatchNorm2d, nn.GroupNorm)):
                nn.init.constant_(m.weight, 1)
                nn.init.constant_(m.bias, 0)

        # Zero-initialize the last BN in each residual branch,
        # so that the residual branch starts with zeros, and each residual block behaves like an identity.
        # This improves the model by 0.2~0.3% according to https://arxiv.org/abs/1706.02677
        if zero_init_residual:
            for m in self.modules():
                if isinstance(m, Bottleneck):
                    nn.init.constant_(m.bn3.weight, 0)
                elif isinstance(m, BasicBlock):
                    nn.init.constant_(m.bn2.weight, 0)

    def _make_layer(self, block, planes, blocks, stride=1, dilate=False):
        norm_layer = self._norm_layer
        downsample = None
        previous_dilation = self.dilation
        if dilate:
            self.dilation *= stride
            stride = 1
        if stride != 1 or self.inplanes != planes * block.expansion:
            downsample = nn.Sequential(
                conv1x1(self.inplanes, planes * block.expansion, stride),
                norm_layer(planes * block.expansion),
            )

        layers = []
        layers.append(block(self.inplanes, planes, stride, downsample, self.groups,
                            self.base_width, previous_dilation, norm_layer))
        self.inplanes = planes * block.expansion
        for _ in range(1, blocks):
            layers.append(block(self.inplanes, planes, groups=self.groups,
                                base_width=self.base_width, dilation=self.dilation,
                                norm_layer=norm_layer))

        return nn.Sequential(*layers)

    def _forward_impl(self, x):
        # See note [TorchScript super()]
        
        x = self.conv1(x)
        x = self.bn1(x)
        x = self.relu(x)
        x = self.maxpool(x)

        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)
        x = self.layer4(x)

        x = self.avgpool(x)
        x = torch.flatten(x, 1)        
        embedding = x
        x = self.fc(embedding)
           
#         return x
        return x, embedding

    def forward(self, x):
        return self._forward_impl(x)

def ResNet50(img_channels=3,num_classes=1000):
     """ResNet-50 model from
     `"Deep Residual Learning for Image Recognition" <https://arxiv.org/pdf/1512.03385.pdf>`_
     Args:
         pretrained (bool): If True, returns a model pre-trained on ImageNet
         progress (bool): If True, displays a progress bar of the download to stderr
     """
     return ResNet(BasicBlock, [3, 4, 6, 3], img_channels,num_classes)

'''
def resnext101_32x8d(pretrained=False, progress=True, **kwargs):
    r"""ResNeXt-101 32x8d model from
    `"Aggregated Residual Transformation for Deep Neural Networks" <https://arxiv.org/pdf/1611.05431.pdf>`_
    Args:
        pretrained (bool): If True, returns a model pre-trained on ImageNet
        progress (bool): If True, displays a progress bar of the download to stderr
    """
    kwargs['groups'] = 32
    kwargs['width_per_group'] = 8
    return _resnet('resnext101_32x8d', Bottleneck, [3, 4, 23, 3],
                   pretrained, progress, **kwargs)
    
def _resnet(arch, block, layers, pretrained, progress, **kwargs):
    model = ResNet(block, layers, **kwargs)
    if pretrained:
        state_dict = load_state_dict_from_url(model_urls[arch],
                                              progress=progress)
        model.load_state_dict(state_dict)
    return model
'''
class Bottleneck(nn.Module):
    # Bottleneck in torchvision places the stride for downsampling at 3x3 convolution(self.conv2)
    # while original implementation places the stride at the first 1x1 convolution(self.conv1)
    # according to "Deep residual learning for image recognition"https://arxiv.org/abs/1512.03385.
    # This variant is also known as ResNet V1.5 and improves accuracy according to
    # https://ngc.nvidia.com/catalog/model-scripts/nvidia:resnet_50_v1_5_for_pytorch.

    expansion = 4

    def __init__(self, inplanes, planes, stride=1, downsample=None, groups=1,
                 base_width=64, dilation=1, norm_layer=None):
        super(Bottleneck, self).__init__()
        if norm_layer is None:
            norm_layer = nn.BatchNorm2d
        width = int(planes * (base_width / 64.)) * groups
        # Both self.conv2 and self.downsample layers downsample the input when stride != 1
        self.conv1 = conv1x1(inplanes, width)
        self.bn1 = norm_layer(width)
        self.conv2 = conv3x3(width, width, stride, groups, dilation)
        self.bn2 = norm_layer(width)
        self.conv3 = conv1x1(width, planes * self.expansion)
        self.bn3 = norm_layer(planes * self.expansion)
        self.relu = nn.ReLU(inplace=True)
        self.downsample = downsample
        self.stride = stride

    def forward(self, x):
        identity = x

        out = self.conv1(x)
        out = self.bn1(out)
        out = self.relu(out)

        out = self.conv2(out)
        out = self.bn2(out)
        out = self.relu(out)

        out = self.conv3(out)
        out = self.bn3(out)

        if self.downsample is not None:
            identity = self.downsample(x)

        out += identity
        out = self.relu(out)

        return out

In [12]:
'''
For BERT
'''
from transformers import BertForSequenceClassification, AdamW, BertConfig
from transformers import get_linear_schedule_with_warmup

**Device**

In [13]:
os.environ["CUDA_VISIBLE_DEVICES"] = "0"
#gpu_ids = [7,6]
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

cuda


**Dataloading Scheme**

In [14]:
import os
os.getcwd()

'/content/drive/MyDrive/DL_PROJECT'

In [15]:
trainlist = '/content/drive/My Drive/DL_PROJECT/hateful_memes/hateful_memes/train.jsonl'
validlist = '/content/drive/My Drive/DL_PROJECT/hateful_memes/hateful_memes/dev_seen.jsonl'

In [16]:
'''
Train Dataloader
''' 
#nw = min([os.cpu_count() // world_size, batch_size if batch_size > 1 else 0, max_worker])
#prime_dict = create_prime_dict(trainlist)

train_dataset = mydataset(trainlist,name='train')   #mydataset_captioning(trainlist,name='train')          
train_dataloader = data.DataLoader(train_dataset, shuffle= True, batch_size = 8, num_workers=16,pin_memory=True)


'''
Validation Dataloader
''' 
validation_dataset = mydataset(validlist, name='valid')  #mydataset_captioning(validlist, name='valid')         
validation_dataloader = data.DataLoader(validation_dataset, shuffle=False, batch_size = 8, num_workers=16,pin_memory=True)

/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:2190: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,
/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 16 worker processes in total. Our suggested max number of worker in current system is 4, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


**Model Definition**


In [17]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
def conv1x1(in_planes, out_planes, stride=1):
    """1x1 convolution"""
    return nn.Conv2d(in_planes, out_planes, kernel_size=1, stride=stride, bias=False)
def conv3x3(in_planes, out_planes, stride=1, groups=1, dilation=1):
    """3x3 convolution with padding"""
    return nn.Conv2d(in_planes, out_planes, kernel_size=3, stride=stride,
                     padding=dilation, groups=groups, bias=False, dilation=dilation)
    
'''
Model1 ResNet50
'''
Image_model = ResNet(Bottleneck, [3, 4, 6, 3], num_classes = 2)
Image_model = nn.DataParallel(Image_model).to(device)


'''
#Load saved model from checkpoint

model1_name = 'Resnet50'
model1_path = './saved_model_checkpoints/'+model1_name

checkpoint1 = torch.load(model1_path)
Image_model.load_state_dict(checkpoint1['model_state_dict'])

Image_model.to(device)
'''
'''
Model1 ResNeXt101_32x8d

#Image_model = ResNet(Bottleneck, [3, 4, 6, 3], num_classes = 2)
Image_model = resnext101_32x8d()

Image_model.fc = nn.Sequential(
    nn.Linear(Image_model.fc.in_features, 2)
    )

Image_model = nn.DataParallel(Image_model).to(device)



#Load saved model from checkpoint

#model1_name = 'ResneXt101_32x8d'
#model1_path = './saved_model_checkpoints/'+model1_name

#checkpoint1 = torch.load(model1_path)
#Image_model.load_state_dict(checkpoint1['model_state_dict'])
#Image_model.to(device)
'''

"\nModel1 ResNeXt101_32x8d\n\n#Image_model = ResNet(Bottleneck, [3, 4, 6, 3], num_classes = 2)\nImage_model = resnext101_32x8d()\n\nImage_model.fc = nn.Sequential(\n    nn.Linear(Image_model.fc.in_features, 2)\n    )\n\nImage_model = nn.DataParallel(Image_model).to(device)\n\n\n\n#Load saved model from checkpoint\n\n#model1_name = 'ResneXt101_32x8d'\n#model1_path = './saved_model_checkpoints/'+model1_name\n\n#checkpoint1 = torch.load(model1_path)\n#Image_model.load_state_dict(checkpoint1['model_state_dict'])\n#Image_model.to(device)\n"

In [18]:
'''
Model 2 BERT

Load BertForSequenceClassification, the pretrained BERT model with a single linear classification layer on top.
''' 

Text_model = BertForSequenceClassification.from_pretrained(
    "bert-base-uncased", 
    num_labels = 2,   
    output_attentions = False, 
    output_hidden_states = True
)

Text_model = nn.DataParallel(Text_model).to(device)



Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

In [34]:
model2_name = 'BERT_basic'
model_path = '/content/drive/My Drive/DL_PROJECT/' +model2_name

In [35]:
'''
Load saved model from checkpoint
'''
checkpoint2 = torch.load(model2_path)
Text_model.load_state_dict(checkpoint2['model_state_dict'])

Text_model.to(device)

FileNotFoundError: ignored

In [22]:
'''
Fusion
'''
'''
class FusionNet(nn.Module):
    
    def __init__(self, num_classes, drop_prob = 0.1):
        super(FusionNet, self).__init__()
        
        self.pooler = nn.Linear(in_features=768, out_features=768)
        
        self.concat = nn.Linear(in_features=768+2048, out_features= 512)
                
        self.classify = nn.Linear(in_features = 512, out_features = num_classes)
        
        
    def forward(self, text_features, image_features):

        x =(text_features+image_features)/2

        return x

'''
#Fusion - Image Features, Text Features and Captions generated by our Captioning model

class FusionNet(nn.Module):
    
    def __init__(self, num_classes, drop_prob = 0.1):
        super(FusionNet, self).__init__()
        
        self.concat = nn.Linear(in_features=768+2048, out_features= 512)
        
        self.bn = nn.BatchNorm1d(512)
        self.bn1 = nn.BatchNorm1d(768)
        self.bn2 = nn.BatchNorm1d(2048)
        #self.bn3 = nn.BatchNorm1d(768)

        
        self.dropout = nn.Dropout(drop_prob)
        
        self.classify = nn.Linear(in_features = 512, out_features = num_classes)
        
        
    def forward(self, text_features, image_features): #, caption_features):

        text_features = self.bn1(text_features)
        image_features = self.bn2(image_features)
        #caption_features = self.bn3(caption_features)

        fused_input =  torch.cat((text_features, image_features), dim=1)
        
        x = self.concat(fused_input)
        x = F.relu(self.bn(x))        
        
        x = F.relu(self.classify(x)) 

        return x


In [49]:
train_loss= []
v_loss = []
v_acc = []

def train(image_model,text_model,fusion_model,data_loader,test_loader,criterion,optimizer, lr_scheduler, modelpath, writer, device, epochs):
    
    fusion_model.train()

    for epoch in range(epochs):
        avg_loss = 0.0
                
        
        for batch_num, (feats, captions, input_id, attention_masks, target) in enumerate(data_loader):
            
            feats, target = feats.to(device), target.to(device)
            input_ids, attention_masks = input_id.to(device), attention_masks.to(device)
               
            '''
            Compute ResNet Features
            '''
            out, image_features = image_model(feats) 
           
                            
            '''
            Compute BERT Features
            Take hidden state corresponding to [CLS] token from the final transformer
            '''
            output_dictionary = text_model(input_ids, 
                                           token_type_ids=None, 
                                           attention_mask=attention_masks, 
                                           labels=target,
                                           return_dict = True)
            
            text_features = output_dictionary.hidden_states[12][:,0,:]
            
            

            
            
            '''
            Compute Classification Output and loss from Fusion model
            '''
            output = fusion_model(text_features, image_features)
            loss = criterion(output, target)

            
                   
            '''
            Take Step
            '''                    
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            
            avg_loss += loss.item()


#             '''
#             linear_schedule_with_warmup take step after each batch
#             '''
#             lr_scheduler.step()
            
                        
#             if batch_num % 100 == 99:
#                 print('loss', avg_loss/100)
                
            del feats
            del captions
            del input_ids
            del attention_masks
            del target
            del loss
            
            
        training_loss = avg_loss/len(data_loader)
       
        print('Epoch: ', epoch+1)            
        print('training loss = ', training_loss)
        train_loss.append(training_loss)

        
        
        '''
        Learning rate scheduler
        '''
        lr_scheduler.step()
            
            
        '''
        Check performance on validation set after an Epoch
        '''
        
        valid_loss, top1_acc= test_classify(image_model, text_model, fusion_model, test_loader, criterion, device)
        print('Validation Loss: {:.4f}\tTop 1 Validation Accuracy: {:.4f}'.format(valid_loss, top1_acc))
        v_loss.append(valid_loss)
        v_acc.append(top1_acc)

        '''
        Logs
        '''
        writer.add_scalar("Loss/train", training_loss, epoch)            
        writer.add_scalar('Loss/Validation', valid_loss, epoch)
        writer.add_scalar('Accuracy/Validation', top1_acc, epoch)

        
        #save fusion model checkpoint after every epoch
        '''
        torch.save({
            'model_state_dict': fusion_model.state_dict(),
            'optimizer_state_dict': optimizer.state_dict(),
            'Training_Loss_List':train_loss,
            'Validation_Loss_List':v_loss,
            'Validation_Accuracy_List': v_acc,
            'Epoch':epoch,
            'lr_scheduler': lr_scheduler.state_dict() 

            }, modelpath)
        '''

        
'''
Returns Loss and top1 accuracy on test/validation set
'''
def test_classify(image_model, text_model, fusion_model, test_loader, criterion, device):
    fusion_model.eval()
    test_loss = []
    top1_accuracy = 0
    total = 0

    for batch_num, (feats, captions, input_id, attention_masks, target) in enumerate(test_loader):
        
        feats, target = feats.to(device), target.to(device)
        input_ids, attention_masks = input_id.to(device), attention_masks.to(device)
        
        
        '''
        Compute ResNet Features
        '''
        out, image_features = image_model(feats) 

        
        
        '''
        Compute BERT Features
        '''
        output_dictionary = text_model(input_ids, 
                                       token_type_ids=None, 
                                       attention_mask=attention_masks, 
                                       labels=target,
                                       return_dict = True)

        text_features = output_dictionary.hidden_states[12][:,0,:]
        
        
        '''
        Compute Classification Output and loss from Fusion model
        '''
        output = fusion_model(text_features, image_features)
        loss = criterion(output, target)

            
        test_loss.extend([loss.item()]*feats.size()[0])
        
        
        
        '''
        Prediction
        '''
        
        predictions = F.softmax(output, dim=1)
        
        _, top1_pred_labels = torch.max(predictions,1)
        top1_pred_labels = top1_pred_labels.view(-1)
        
        top1_accuracy += torch.sum(torch.eq(top1_pred_labels, target)).item()
        
        
        total += len(target)
        
        del feats
        del captions
        del input_ids
        del attention_masks
        del target
        del loss
            
    fusion_model.train()
    return np.mean(test_loss), top1_accuracy/total

In [24]:
Fusion_model = FusionNet(num_classes = 2 , drop_prob = 0.1)
Fusion_model = nn.DataParallel(Fusion_model).to(device)

In [33]:
model_name = 'Late Fusion Model'
model_path = '/content/drive/My Drive/DL_PROJECT/' +model_name

**Hyperparameters**

In [36]:
'''
Loss Function
'''
criterion = nn.CrossEntropyLoss()


'''
Optimizer
'''
optimizer = torch.optim.SGD(Fusion_model.parameters(), lr=0.01, weight_decay=1e-4, momentum=0.9)
# optimizer = AdamW(Fusion_model.parameters(), lr = 2e-3, eps = 1e-8)


'''
Number of training epochs.
'''
num_Epochs = 2


# '''
# OneCycleLR
# '''
# max_lr = 0.05
# lr_scheduler = torch.optim.lr_scheduler.OneCycleLR(optimizer, max_lr, total_steps=None, epochs=num_Epochs, steps_per_epoch=len(train_dataloader), pct_start=0.3, anneal_strategy='cos', cycle_momentum=True, base_momentum=0.85, max_momentum=0.95, div_factor=25.0, final_div_factor=10000.0, last_epoch=-1)


lr_scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size= 4, gamma = 0.1)

# lr_scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max, eta_min=0, last_epoch=-1)

In [39]:
'''
Load saved model from checkpoint
'''
#Fusion_model, optimizer, lr_scheduler, train_loss, v_loss, v_acc, epoch = load(model_path, Fusion_model, optimizer, lr_scheduler)

'\nLoad saved model from checkpoint\n'

In [50]:
#torch.cuda.memory_summary(device=None, abbreviated=False)
torch.cuda.empty_cache()
from torch.utils.tensorboard import SummaryWriter
writer = SummaryWriter(model_name)

train(Image_model, Text_model, Fusion_model, train_dataloader, validation_dataloader, criterion, optimizer, lr_scheduler, model_path, writer, device, epochs = num_Epochs)

writer.flush()
writer.close()

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 16 worker processes in total. Our suggested max number of worker in current system is 4, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


Epoch:  1
training loss =  0.6931471824645996
Validation Loss: 0.6931	Top 1 Validation Accuracy: 0.5060
Epoch:  2
training loss =  0.6931471824645996
Validation Loss: 0.6931	Top 1 Validation Accuracy: 0.5060


**Evaluate**

**Predict on Test and generate output.csv**

**Test Dataloader**

In [41]:
testlist = '/content/drive/My Drive/DL_PROJECT/hateful_memes/hateful_memes/test_seen.jsonl'

test_dataset = mytestdataset(testlist, name='test')          
test_dataloader = data.DataLoader(test_dataset, shuffle= False, batch_size = 32, num_workers=8,pin_memory=True)

/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:2190: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,
/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 8 worker processes in total. Our suggested max number of worker in current system is 4, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


In [42]:
test_classify(Image_model, Text_model, Fusion_model, validation_dataloader, criterion, device)

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 16 worker processes in total. Our suggested max number of worker in current system is 4, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


(0.6931471824645996, 0.506)

In [ ]:
from predict import predict
predict(image_model, text_model, fusion_model, test_dataloader, device)

ModuleNotFoundError: ignored